In [ ]:
%load_ext autoreload
%autoreload 2

import pandas_profiling
%pylab inline

import pandas as pd
import geopandas as gp

import seaborn as sns
sns.set(color_codes=True)
import numpy as np

from pathlib import Path
import itertools

from analysis.settings.settings import DevelopmentConfig
c = DevelopmentConfig

# open data integration
Cleaning shape files delivered as open data from http://www.laerminfo.at/laermkarten/methoden/inspire.html

In [ ]:
%ls ../data/

In [ ]:
def iter_dirs(directory_in_str, glob):
    pathlist = Path(directory_in_str).glob(glob)
    for path in pathlist:
        yield str(path)
        
def parse_attributes_from_path(path):
    file_name = path.split('/')[-1]
    elements = file_name.split('_')
    result = {}
    result['year'] = elements[0]
    result['kind'] = elements[1]
    result['timing'] = elements[2]
    result['state'] = elements[-1].split('.')[0]
    return result

def add_columns_to_df(df, items):
    for key, value in items.items():
        df[key] = value
    return df

In [ ]:
paths = iter_dirs(c.BASE_PATH, '**/*.shp')
paths = itertools.islice(paths, 2)
#[parse_attributes_from_path(path) for path in paths]
tmp_appended_data = []
for path in paths:
    print(path)
    attributes_from_filenname = parse_attributes_from_path(path)
    df = gp.read_file(path)
    df = add_columns_to_df(df, attributes_from_filenname)
    tmp_appended_data.append(df)

In [ ]:
df = pd.concat(tmp_appended_data, axis=0)
df = df.reset_index(drop=True)
display(df.head())
df.shape

## Write output files
Geopackage is a nice standard and a bit more structured than CSV.

In [ ]:
#df.to_csv(c.BASE_PATH + 'noise_pollution.gzip.csv', index=False, compression='gzip')
# is a bit smaller in file size

df.to_file(c.BASE_PATH + 'noise_pollution.gpkg', driver="GPKG")